In [15]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import random

# Q: So the AgentState will never stop unless the outcome is AgentFinish?

class AgentState(TypedDict):
   # The input string
   input: str
   # The list of previous messages in the conversation
   chat_history: list[BaseMessage]
   # The outcome of a given call to the agent
   # Needs `None` as a valid type, since this is what this will start as
   agent_outcome: Union[AgentAction, AgentFinish, None]
   # List of actions and corresponding observations
   # Here we annotate this with `operator.add` to indicate that operations to
   # this state should be ADDED to the existing values (not overwrite it)
   intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100."""
    return random.randint(0, 100)


In [17]:
tools = [to_lower_case,random_number_maker]

In [7]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm,
                                               tools,
                                               prompt)

In [12]:
from langchain.prompts import ChatPromptTemplate

# Create a chat prompt template
chat_template = ChatPromptTemplate.from_messages([
    ("human", "What is the capital of {country}?"),
    ("ai", "The capital of {country} is {capital}.")
])

# Invoke the template with input values
chat_prompt = chat_template.invoke({"country": "France", "capital": "Paris"})
# chat_prompt = chat_template.format_messages(conutry="France", capital="Paris")

# Get the formatted chat messages
formatted_messages = chat_prompt.to_messages()

# Print the formatted messages
formatted_messages


[HumanMessage(content='What is the capital of France?'),
 AIMessage(content='The capital of France is Paris.')]

In [18]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
sequence = runnable_1 | runnable_2
# Or equivalently:
# sequence = RunnableSequence(first=runnable_1, last=runnable_2)
sequence.invoke(1)
await sequence.ainvoke(1)

# sequence.batch([1, 2, 3])
# await sequence.abatch([1, 2, 3])

4

In [19]:
runnable_1

RunnableLambda(add_one)

In [21]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

In [23]:
# agent_runnable.invoke gets you reponse from the agent
# tool_executor.invoke gets you output from the tools called
# Define the agent/graph
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data['agent_outcome']
    # Execute the tool
    output = tool_executor.invoke(agent_action)
    print(f"The agent action is {agent_action}")
    print(f"The tool result is: {output}")
    # Return the output
    return {"intermediate_steps": [(agent_action, str(output))]}

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

In [24]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [26]:
workflow.branches

defaultdict(list,
            {'agent': [Branch(condition=<function should_continue at 0x129ce5940>, ends={'continue': 'action', 'end': '__end__'})]})

In [28]:
workflow.nodes, workflow.edges

({'agent': RunnableLambda(run_agent), 'action': RunnableLambda(execute_tools)},
 {('action', 'agent')})

In [29]:
workflow.channels

{'input': <langgraph.channels.last_value.LastValue at 0x12893c410>,
 'chat_history': <langgraph.channels.last_value.LastValue at 0x129aa6c10>,
 'agent_outcome': <langgraph.channels.last_value.LastValue at 0x129aa6690>,
 'intermediate_steps': <langgraph.channels.binop.BinaryOperatorAggregate at 0x129aa6b50>}

In [30]:
inputs = {"input": "give me a random number and then write in words and make it lower case.", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='random_number', tool_input={'input': 'random number'}, log="\nInvoking: `random_number` with `{'input': 'random number'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random number"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call'})])}
----
The agent action is tool='random_number' tool_input={'input': 'random number'} log="\nInvoking: `random_number` with `{'input': 'random number'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random number"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call'})]
The tool result is: 35
{'intermediate_steps': [(AgentActionMessageLog(tool='random_number', tool_input={'input': 'random number'}, log="\nInvoking: `random_number` with `{'input': 'random number'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_ca